#Install Transformers

In [ ]:
!pip install datasets
!pip install -U sentence-transformers
!pip install transformers
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=202180aecf62dfe6f48ca63f9b340cabaebdfe370e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import libraries

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
from datasets import load_dataset
import pandas as pd
import numpy as np

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 10
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name

#Loading Ontology Data Files for embedding

In [ ]:
sourceOntBL = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/sourceOntBL.csv', header=None)
sourceOntBL.rename(columns={0: "entity", 1: "text"}, inplace=True)
sourceOntKI = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/sourceOntKI.csv', header=None)
sourceOntKI.rename(columns={0: "entity", 1: "text"}, inplace=True)
targetOntBL = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/targetOntBL.csv', header=None)
targetOntBL.rename(columns={0: "entity", 1: "text"}, inplace=True)
targetOntKI = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/targetOntKI.csv', header=None)
targetOntKI.rename(columns={0: "entity", 1: "text"}, inplace=True)
sourceOntKS = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/sourceOntKISym.csv', header=None)
sourceOntKS.rename(columns={0: "entity", 1: "text"}, inplace=True)
targetOntKS = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/targetOntKISym.csv', header=None)
targetOntKS.rename(columns={0: "entity", 1: "text"}, inplace=True)
sourceOntKJ = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/sourceOntKIJoin.csv', header=None)
sourceOntKJ.rename(columns={0: "entity", 1: "text"}, inplace=True)
targetOntKJ = pd.read_csv('/content/drive/MyDrive/Thesis/OMIM2ORDO/targetOntKIJoin.csv', header=None)
targetOntKJ.rename(columns={0: "entity", 1: "text"}, inplace=True)

sourceOntBL.shape, sourceOntKI.shape, targetOntBL.shape, targetOntKI.shape, sourceOntKS.shape, targetOntKS.shape, sourceOntKJ.shape, targetOntKJ.shape

((9642, 2),
 (9755, 2),
 (8838, 2),
 (17150, 2),
 (9755, 2),
 (17150, 2),
 (9642, 2),
 (8838, 2))

#Load the Baseline Model - Model = BL

In [ ]:
model_name = 'FremyCompany/BioLORD-STAMB2-v1'
modelBL = SentenceTransformer(model_name)
#embeddings = model.encode(sentences)
print(modelBL)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


#Embeddings using BL Model, OntFiles - BL, KI, KS

BLBL, BLKI, BLKS

In [ ]:
embeddingsSrcBLBL = modelBL.encode(sourceOntBL["text"])
embeddingsTgtBLBL = modelBL.encode(targetOntBL["text"])
"""embeddingsSrcBLKI = modelBL.encode(sourceOntKI["text"])
embeddingsTgtBLKI = modelBL.encode(targetOntKI["text"])
embeddingsSrcBLKS = modelBL.encode(sourceOntKS["text"])
embeddingsTgtBLKS = modelBL.encode(targetOntKS["text"])"""

#sourceOntBL.shape, embeddingsSrcBLBL.shape, targetOntBL.shape, embeddingsTgtBLBL.shape, sourceOntKS.shape, embeddingsSrcBLKI.shape, targetOntKS.shape, embeddingsTgtBLKI.shape

'embeddingsSrcBLKI = modelBL.encode(sourceOntKI["text"])\nembeddingsTgtBLKI = modelBL.encode(targetOntKI["text"])\nembeddingsSrcBLKS = modelBL.encode(sourceOntKS["text"])\nembeddingsTgtBLKS = modelBL.encode(targetOntKS["text"])'

In [ ]:
#"Entity«Name|[vector]«Name|[vector] \n"

srcBLBL = pd.DataFrame()
srcBLBL['entity-name'] = pd.DataFrame(sourceOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcBLBL['vector'] = pd.DataFrame(embeddingsSrcBLBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcBLBL['combined'] = pd.DataFrame(srcBLBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtBLBL = pd.DataFrame()
tgtBLBL['entity-name'] = pd.DataFrame(targetOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtBLBL['vector'] = pd.DataFrame(embeddingsTgtBLBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtBLBL['combined'] = pd.DataFrame(tgtBLBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

"""srcBLKI = pd.DataFrame()
srcBLKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcBLKI['vector'] = pd.DataFrame(embeddingsSrcBLKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcBLKI['combined'] = pd.DataFrame(srcBLKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtBLKI = pd.DataFrame()
tgtBLKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtBLKI['vector'] = pd.DataFrame(embeddingsTgtBLKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtBLKI['combined'] = pd.DataFrame(tgtBLKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

srcBLKS = pd.DataFrame()
srcBLKS['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcBLKS['vector'] = pd.DataFrame(embeddingsSrcBLKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcBLKS['combined'] = pd.DataFrame(srcBLKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtBLKS = pd.DataFrame()
tgtBLKS['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtBLKS['vector'] = pd.DataFrame(embeddingsTgtBLKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtBLKS['combined'] = pd.DataFrame(tgtBLKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'"""


"srcBLKI = pd.DataFrame()\nsrcBLKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\nsrcBLKI['vector'] = pd.DataFrame(embeddingsSrcBLKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\nsrcBLKI['combined'] = pd.DataFrame(srcBLKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'\n\ntgtBLKI = pd.DataFrame()\ntgtBLKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\ntgtBLKI['vector'] = pd.DataFrame(embeddingsTgtBLKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\ntgtBLKI['combined'] = pd.DataFrame(tgtBLKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'\n\nsrcBLKS = pd.DataFrame()\nsrcBLKS['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\nsrcBLKS['vector'] = pd.DataFrame(embeddingsSrcBLKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\nsrcBLKS['combined'] 

In [ ]:
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcBLBL.txt', srcBLBL['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtBLBL.txt', tgtBLBL['combined'].values, fmt='%s')
"""np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcBLKI.txt', srcBLKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtBLKI.txt', tgtBLKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcBLKS.txt', srcBLKS['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtBLKS.txt', tgtBLKS['combined'].values, fmt='%s')"""

"np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcBLKI.txt', srcBLKI['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtBLKI.txt', tgtBLKI['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcBLKS.txt', srcBLKS['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtBLKS.txt', tgtBLKS['combined'].values, fmt='%s')"

#Load the Model = FT



In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFT = SentenceTransformer(model_save_path)
#embeddings = model.encode(sentences)
print(modelFT)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


#Embeddings using FT model, OntFiles - BL, KI, KS

FTBL, FTKI, FTKS

In [ ]:
embeddingsSrcFTBL = modelFT.encode(sourceOntBL["text"])
embeddingsTgtFTBL = modelFT.encode(targetOntBL["text"])
"""embeddingsSrcFTKI = modelFT.encode(sourceOntKI["text"])
embeddingsTgtFTKI = modelFT.encode(targetOntKI["text"])
embeddingsSrcFTKS = modelFT.encode(sourceOntKS["text"])
embeddingsTgtFTKS = modelFT.encode(targetOntKS["text"])"""
#sourceOntBL.shape, embeddingsSrcFTBL.shape, targetOntBL.shape, embeddingsTgtFTBL.shape, sourceOntKI.shape, embeddingsSrcFTKI.shape, targetOntKI.shape, embeddingsTgtFTKI.shape, sourceOntKS.shape, embeddingsSrcFTKS.shape, targetOntKS.shape, embeddingsTgtFTKS.shape

'embeddingsSrcFTKI = modelFT.encode(sourceOntKI["text"])\nembeddingsTgtFTKI = modelFT.encode(targetOntKI["text"])\nembeddingsSrcFTKS = modelFT.encode(sourceOntKS["text"])\nembeddingsTgtFTKS = modelFT.encode(targetOntKS["text"])'

In [ ]:
#"Entity«Name|[vector]«Name|[vector] \n"

srcFTBL = pd.DataFrame()
srcFTBL['entity-name'] = pd.DataFrame(sourceOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFTBL['vector'] = pd.DataFrame(embeddingsSrcFTBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFTBL['combined'] = pd.DataFrame(srcFTBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFTBL = pd.DataFrame()
tgtFTBL['entity-name'] = pd.DataFrame(targetOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFTBL['vector'] = pd.DataFrame(embeddingsTgtFTBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFTBL['combined'] = pd.DataFrame(tgtFTBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

"""srcFTKI = pd.DataFrame()
srcFTKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFTKI['vector'] = pd.DataFrame(embeddingsSrcFTKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFTKI['combined'] = pd.DataFrame(srcFTKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFTKI = pd.DataFrame()
tgtFTKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFTKI['vector'] = pd.DataFrame(embeddingsTgtFTKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFTKI['combined'] = pd.DataFrame(tgtFTKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

srcFTKS = pd.DataFrame()
srcFTKS['entity-name'] = pd.DataFrame(sourceOntKS).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFTKS['vector'] = pd.DataFrame(embeddingsSrcFTKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFTKS['combined'] = pd.DataFrame(srcFTKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFTKS = pd.DataFrame()
tgtFTKS['entity-name'] = pd.DataFrame(targetOntKS).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFTKS['vector'] = pd.DataFrame(embeddingsTgtFTKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFTKS['combined'] = pd.DataFrame(tgtFTKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'"""

"srcFTKI = pd.DataFrame()\nsrcFTKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\nsrcFTKI['vector'] = pd.DataFrame(embeddingsSrcFTKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\nsrcFTKI['combined'] = pd.DataFrame(srcFTKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'\n\ntgtFTKI = pd.DataFrame()\ntgtFTKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\ntgtFTKI['vector'] = pd.DataFrame(embeddingsTgtFTKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\ntgtFTKI['combined'] = pd.DataFrame(tgtFTKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'\n\nsrcFTKS = pd.DataFrame()\nsrcFTKS['entity-name'] = pd.DataFrame(sourceOntKS).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)\nsrcFTKS['vector'] = pd.DataFrame(embeddingsSrcFTKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)\nsrcFTKS['combined'] 

In [ ]:
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTBL.txt', srcFTBL['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTBL.txt', tgtFTBL['combined'].values, fmt='%s')
"""np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTKI.txt', srcFTKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTKI.txt', tgtFTKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTKS.txt', srcFTKS['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTKS.txt', tgtFTKS['combined'].values, fmt='%s')"""

"np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTKI.txt', srcFTKI['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTKI.txt', tgtFTKI['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTKS.txt', srcFTKS['combined'].values, fmt='%s')\nnp.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTKS.txt', tgtFTKS['combined'].values, fmt='%s')"

#Load the Model = FS

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FS'

In [ ]:
modelFS = SentenceTransformer(model_save_path)
#embeddings = model.encode(sentences)
print(modelFS)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


#Embeddings using FS model, OntFiles - BL, KI, KS

In [ ]:
"""embeddingsSrcFSBL = modelFS.encode(sourceOntBL["text"])
embeddingsTgtFSBL = modelFS.encode(targetOntBL["text"])
embeddingsSrcFSKI = modelFS.encode(sourceOntKI["text"])
embeddingsTgtFSKI = modelFS.encode(targetOntKI["text"])"""
embeddingsSrcFSKS = modelFS.encode(sourceOntKS["text"])
embeddingsTgtFSKS = modelFS.encode(targetOntKS["text"])
#sourceOntBL.shape, embeddingsSrcFSBL.shape, targetOntBL.shape, embeddingsTgtFSBL.shape, sourceOntKI.shape, embeddingsSrcFSKI.shape, targetOntKI.shape, embeddingsTgtFSKI.shape, sourceOntKS.shape, embeddingsSrcFSKS.shape, targetOntKS.shape, embeddingsTgtFSKS.shape

In [ ]:
#"Entity«Name|[vector]«Name|[vector] \n"

"""srcFSBL = pd.DataFrame()
srcFSBL['entity-name'] = pd.DataFrame(sourceOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFSBL['vector'] = pd.DataFrame(embeddingsSrcFSBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFSBL['combined'] = pd.DataFrame(srcFSBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFSBL = pd.DataFrame()
tgtFSBL['entity-name'] = pd.DataFrame(targetOntBL).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFSBL['vector'] = pd.DataFrame(embeddingsTgtFSBL).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFSBL['combined'] = pd.DataFrame(tgtFSBL).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

srcFSKI = pd.DataFrame()
srcFSKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFSKI['vector'] = pd.DataFrame(embeddingsSrcFSKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFSKI['combined'] = pd.DataFrame(srcFSKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFSKI = pd.DataFrame()
tgtFSKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFSKI['vector'] = pd.DataFrame(embeddingsTgtFSKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFSKI['combined'] = pd.DataFrame(tgtFSKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'"""

srcFSKS = pd.DataFrame()
srcFSKS['entity-name'] = pd.DataFrame(sourceOntKS).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFSKS['vector'] = pd.DataFrame(embeddingsSrcFSKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFSKS['combined'] = pd.DataFrame(srcFSKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFSKS = pd.DataFrame()
tgtFSKS['entity-name'] = pd.DataFrame(targetOntKS).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFSKS['vector'] = pd.DataFrame(embeddingsTgtFSKS).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFSKS['combined'] = pd.DataFrame(tgtFSKS).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'


In [ ]:
"""np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFSBL.txt', srcFSBL['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFSBL.txt', tgtFSBL['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFSKI.txt', srcFSKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFSKI.txt', tgtFSKI['combined'].values, fmt='%s')"""
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFSKS.txt', srcFSKS['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFSKS.txt', tgtFSKS['combined'].values, fmt='%s')

# Embeddings using FT model, OntFiles - KD

FTKD (Knowledge injection KI WITHOUT Duplicates)

In [ ]:
# FTKD : FT + KI without duplicates [KD]

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'
modelFT = SentenceTransformer(model_save_path)
#print(modelFT)

sourceOntKD = pd.DataFrame()
targetOntKD = pd.DataFrame()
sourceOntKD["entity"] = sourceOntKI["entity"]
sourceOntKD["text"] = sourceOntKI["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
targetOntKD["entity"] = targetOntKI["entity"]
targetOntKD["text"] = targetOntKI["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFTKD = modelFT.encode(sourceOntKD["text"])
embeddingsTgtFTKD = modelFT.encode(targetOntKD["text"])

srcFTKD = pd.DataFrame()
srcFTKD['entity-name'] = pd.DataFrame(sourceOntKD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFTKD['vector'] = pd.DataFrame(embeddingsSrcFTKD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFTKD['combined'] = pd.DataFrame(srcFTKD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFTKD = pd.DataFrame()
tgtFTKD['entity-name'] = pd.DataFrame(targetOntKD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFTKD['vector'] = pd.DataFrame(embeddingsTgtFTKD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFTKD['combined'] = pd.DataFrame(tgtFTKD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFTKD.txt', srcFTKD['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFTKD.txt', tgtFTKD['combined'].values, fmt='%s')




#FDKD

In [ ]:
# FDKD : FD + KI without duplicates [KD]

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FD'
modelFD = SentenceTransformer(model_save_path)
#print(modelFT)

sourceOntKD = pd.DataFrame()
targetOntKD = pd.DataFrame()
sourceOntKD["entity"] = sourceOntKI["entity"]
sourceOntKD["text"] = sourceOntKI["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
targetOntKD["entity"] = targetOntKI["entity"]
targetOntKD["text"] = targetOntKI["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFDKD = modelFD.encode(sourceOntKD["text"])
embeddingsTgtFDKD = modelFD.encode(targetOntKD["text"])

srcFDKD = pd.DataFrame()
srcFDKD['entity-name'] = pd.DataFrame(sourceOntKD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFDKD['vector'] = pd.DataFrame(embeddingsSrcFDKD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFDKD['combined'] = pd.DataFrame(srcFDKD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFDKD = pd.DataFrame()
tgtFDKD['entity-name'] = pd.DataFrame(targetOntKD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFDKD['vector'] = pd.DataFrame(embeddingsTgtFDKD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFDKD['combined'] = pd.DataFrame(tgtFDKD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFDKD.txt', srcFDKD['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFDKD.txt', tgtFDKD['combined'].values, fmt='%s')




#FCKC

Fine Tune Model = FC,
Training Files = KS (KS + colon replaced with comma),
OntFiles = (KS + colon replaced with comma)

In [ ]:
# FCKC : FC + KS + colon replaced with comma (KC)

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FC'
modelFC = SentenceTransformer(model_save_path)
#print(modelFT)

sourceOntKC = pd.DataFrame()
targetOntKC = pd.DataFrame()
sourceOntKC["entity"] = sourceOntKS["entity"]
sourceOntKC["text"] = sourceOntKS["text"].replace(":",",")
targetOntKC["entity"] = targetOntKS["entity"]
targetOntKC["text"] = targetOntKS["text"].replace(":",",")
#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFCKC = modelFC.encode(sourceOntKC["text"])
embeddingsTgtFCKC = modelFC.encode(targetOntKC["text"])

srcFCKC = pd.DataFrame()
srcFCKC['entity-name'] = pd.DataFrame(sourceOntKC).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFCKC['vector'] = pd.DataFrame(embeddingsSrcFCKC).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFCKC['combined'] = pd.DataFrame(srcFCKC).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFCKC = pd.DataFrame()
tgtFCKC['entity-name'] = pd.DataFrame(targetOntKC).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFCKC['vector'] = pd.DataFrame(embeddingsTgtFCKC).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFCKC['combined'] = pd.DataFrame(tgtFCKC).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFCKC.txt', srcFCKC['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFCKC.txt', tgtFCKC['combined'].values, fmt='%s')




#FSDKSD

(KI with sym colon but no duplicates)

In [ ]:
# FSDKSD : FSD + KS without duplicates [KD]

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FSD'
modelFSD = SentenceTransformer(model_save_path)
#print(modelFT)

sourceOntKSD = pd.DataFrame()
targetOntKSD = pd.DataFrame()
sourceOntKSD["entity"] = sourceOntKS["entity"]
sourceOntKSD["text"] = sourceOntKS["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
targetOntKSD["entity"] = targetOntKS["entity"]
targetOntKSD["text"] = targetOntKS["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFSDKSD = modelFSD.encode(sourceOntKSD["text"])
embeddingsTgtFSDKSD = modelFSD.encode(targetOntKSD["text"])

srcFSDKSD = pd.DataFrame()
srcFSDKSD['entity-name'] = pd.DataFrame(sourceOntKSD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFSDKSD['vector'] = pd.DataFrame(embeddingsSrcFSDKSD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFSDKSD['combined'] = pd.DataFrame(srcFSDKSD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFSDKSD = pd.DataFrame()
tgtFSDKSD['entity-name'] = pd.DataFrame(targetOntKSD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFSDKSD['vector'] = pd.DataFrame(embeddingsTgtFSDKSD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFSDKSD['combined'] = pd.DataFrame(tgtFSDKSD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFSDKSD.txt', srcFSDKSD['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFSDKSD.txt', tgtFSDKSD['combined'].values, fmt='%s')




#FIKI

In [ ]:
# FIKI : FI + KI

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FI'
modelFI = SentenceTransformer(model_save_path)
#print(modelFT)

#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFIKI = modelFI.encode(sourceOntKI["text"])
embeddingsTgtFIKI = modelFI.encode(targetOntKI["text"])

srcFIKI = pd.DataFrame()
srcFIKI['entity-name'] = pd.DataFrame(sourceOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFIKI['vector'] = pd.DataFrame(embeddingsSrcFIKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFIKI['combined'] = pd.DataFrame(srcFIKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFIKI = pd.DataFrame()
tgtFIKI['entity-name'] = pd.DataFrame(targetOntKI).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFIKI['vector'] = pd.DataFrame(embeddingsTgtFIKI).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFIKI['combined'] = pd.DataFrame(tgtFIKI).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFIKI.txt', srcFIKI['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFIKI.txt', tgtFIKI['combined'].values, fmt='%s')




#FJKJ

In [ ]:
# FJKJ : FJ + KJ

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FJ'
modelFJ = SentenceTransformer(model_save_path)
#print(modelFT)

#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFJKJ = modelFJ.encode(sourceOntKJ["text"])
embeddingsTgtFJKJ = modelFJ.encode(targetOntKJ["text"])

srcFJKJ = pd.DataFrame()
srcFJKJ['entity-name'] = pd.DataFrame(sourceOntKJ).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFJKJ['vector'] = pd.DataFrame(embeddingsSrcFJKJ).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFJKJ['combined'] = pd.DataFrame(srcFJKJ).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFJKJ = pd.DataFrame()
tgtFJKJ['entity-name'] = pd.DataFrame(targetOntKJ).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFJKJ['vector'] = pd.DataFrame(embeddingsTgtFJKJ).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFJKJ['combined'] = pd.DataFrame(tgtFJKJ).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFJKJ.txt', srcFJKJ['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFJKJ.txt', tgtFJKJ['combined'].values, fmt='%s')




#FJDKJD

In [ ]:
# FJDKJD : FJD + KJ without duplicates

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FJD'
modelFJD = SentenceTransformer(model_save_path)
#print(modelFT)

sourceOntKJD = pd.DataFrame()
targetOntKJD = pd.DataFrame()
sourceOntKJD["entity"] = sourceOntKJ["entity"]
sourceOntKJD["text"] = sourceOntKJ["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
targetOntKJD["entity"] = targetOntKJ["entity"]
targetOntKJD["text"] = targetOntKJ["text"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
#sourceOntKD, sourceOntKI, targetOntKD, targetOntKI

embeddingsSrcFJDKJD = modelFJD.encode(sourceOntKJD["text"])
embeddingsTgtFJDKJD = modelFJD.encode(targetOntKJD["text"])

srcFJDKJD = pd.DataFrame()
srcFJDKJD['entity-name'] = pd.DataFrame(sourceOntKJD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
srcFJDKJD['vector'] = pd.DataFrame(embeddingsSrcFJDKJD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
srcFJDKJD['combined'] = pd.DataFrame(srcFJDKJD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

tgtFJDKJD = pd.DataFrame()
tgtFJDKJD['entity-name'] = pd.DataFrame(targetOntKJD).apply(lambda row: '«'.join(row.values.astype(str)), axis=1)
tgtFJDKJD['vector'] = pd.DataFrame(embeddingsTgtFJDKJD).apply(lambda row: ','.join(row.values.astype(str)), axis=1)
tgtFJDKJD['combined'] = pd.DataFrame(tgtFJDKJD).apply(lambda row: '|['.join(row.values.astype(str)), axis=1)+ ']'

np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/srcFJDKJD.txt', srcFJDKJD['combined'].values, fmt='%s')
np.savetxt(r'/content/drive/MyDrive/Thesis/OMIM2ORDO/output/tgtFJDKJD.txt', tgtFJDKJD['combined'].values, fmt='%s')


